# Compute the SN

In [1]:
import os
import random
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

import galsim
import astropy
import astropy.units as units
from astropy.table import Table, Column
import cosmology
import sep
import fitsio
from unagi import filters
from unagi import camera

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
plt.rcParams['figure.figsize'] = [9, 6]
plt.rcParams["font.family"] = "serif"
plt.rcParams['text.usetex'] = False
%matplotlib inline

from scipy.special import jn, jn_zeros
from hankel_transform import *
from power_spectra import *
from astropy.cosmology import Planck15 as cosmo
import astropy.units as u
from matplotlib.colors import LogNorm
from scipy.signal import savgol_filter
import scipy.integrate as integrate
cosmo_h=cosmo.clone(H0=70)

In [2]:
# Useful constants
lsun = 3.846e33  # erg/s
pc = 3.085677581467192e18  # in cm
lightspeed = 2.998e18  # AA/s
ckms = 2.998e5  # km/s
jansky_mks = 1e-26
jansky_cgs = 1e-23

#set cosmology
cosmo_for_sn = cosmology.Cosmo(H0=70, omega_m = 0.3)

#Get HSC camera properties
hsc_cam = camera.Camera()
#Collecting area of telescope in cm^2
area = 469530 #HSC
#pizelsize : fix for now to 0.18"
pixel_scale = 0.17 # HSC arcsec/pixel
#image size
length = 64 #pixels
#which filter to use (0 for nb, 1 for HSC_r2)
filter_number = 0
#fixed number of nights
Nnights = 10
#hours per night: assume 8
hours_night = 8
#assume size of imager in deg^2
imager_size = 1.76 
# Option: use shape noise only (shapenoiseonly=1), or all sources of noise
shapenoiseonly = 0
# HSC wide
z_source=np.atleast_1d([0.81])   # mean redshift of sources
sigma_gamma=0.28
n_s_arcmin=18.5 #n sources per arcmin^2
# Range of lens M*
Ms_min = 8.0
Ms_max = 9.0

In [3]:
datapath = "/Users/yifei/work/dwarf/computesn"

# Read in the COSMOS GALAXY SMF
SMF_prof=np.genfromtxt('./cosmos2015_dic2017_smf_z01-04_STY0.txt',names=('log_m','log_phi','log_phi_inf','log_phi_sup'))
log_m=SMF_prof['log_m']
log_phi=SMF_prof['log_phi']

# Read in the sky spectra
spec_sky = os.path.join(datapath, 'share/SEDs/SKY_OPTICAL_SPECTRUM_TEMPLATE_GUNN.txt')
spec_sky = np.genfromtxt(spec_sky,names=('sky_wave','sky_flux_1','sky_flux_2'))
sky_wave = spec_sky['sky_wave']
sky_flux = spec_sky['sky_flux_2']*271725 #fit this number to match HSC ETC sky value
spec_sky_func = interp1d(sky_wave, sky_flux, kind=3)

# Corresponding DS file
DS_prof=np.genfromtxt('./dwarf_smooth_model_8_9.txt',names=('rp','DS'))

#integrating the SMF
SMF = interp1d(log_m, log_phi, kind=3)
x_SMF = np.linspace(0,13,10000)
Phi_interp1d = SMF(x_SMF)
SMF_new = interp1d(x_SMF, pow(10,Phi_interp1d), kind=3)
phiintegral = integrate.quad(lambda x: SMF_new(x), Ms_min, Ms_max)[0]

In [4]:
# Set up the R binning in DS
dlogr=np.gradient(np.log10(DS_prof['rp']))[0]
rmin=0.005 #10**(np.log10(DS_prof['rp'][0])-dlogr/2)
rmax=1.9 #10**(np.log10(DS_prof['rp'][-1])+dlogr/2)
nbins=len(DS_prof['rp'])

rp_bins=np.logspace(np.log10(rmin),np.log10(rmax),nbins+1)

#Set up a new set of parameters for power spectra and correlation functions
PS=Power_Spectra()
PS.pk_params['kmax']=520
PS.pk_params['kmin']=1e-2
PS.pk_params['non_linear']=1
rmin=0.005
rmax=1.9

cosmo_params=dict({'h':cosmo.h,'Omb':cosmo.Ob0,'Omd':cosmo.Odm0,'Om':cosmo.Om0,
                'As':2.14e-09,'mnu':cosmo.m_nu[-1].value,'Omk':cosmo.Ok0,'tau':0.06,'ns':0.965,
                  'w':-1,'wa':0})

#Setting up the Hankel Transform
#This deals with the interpolation over the Bessel Functions
#This part is slower. But only needs to be run once. 
#If you only need wgg, set j_nu=[0]. For wg+ (or \Delta\Sigma) use j_nu=[2]
%time HT=hankel_transform(rmin=rmin,rmax=rmax,kmax=PS.pk_params['kmax'],j_nu=[2],n_zeros=80000,kmin=PS.pk_params['kmin'],prune_r=0)
%time HT_inv=hankel_transform(rmin=PS.pk_params['kmin'],rmax=PS.pk_params['kmax'],kmax=rmax,j_nu=[2],n_zeros=80000,kmin=rmin,prune_r=50)
#HT=hankel_transform(rmin=1,rmax=rmax,kmax=1,j_nu=[0,2],n_zeros=2800,kmin=1.e-2)#quick test... inaccurate

changed kmax to 1027.1244603681366  to cover rmin
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 81000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 82000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 83000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 84000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 85000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 86000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 87000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 88000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 89000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 90000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 91000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 92000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 93000
j-nu= 2  not enough zeros t

j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 106000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 107000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 108000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 109000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 110000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 111000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 112000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 113000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 114000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 115000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 116000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 117000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to 118000
j-nu= 2  not enough zeros to cover kmin, increasing by  1000  to

In [5]:
CCD_qe = interp1d(hsc_cam.qe[:, 0], hsc_cam.qe[:, 1], kind=3)
x_wave = np.linspace(4000,10000,10000)
CCD_qe_interp1d = CCD_qe(x_wave)

popt = interp1d(hsc_cam.popt2[:, 0], hsc_cam.popt2[:, 1], kind=3)
x_wave = np.linspace(4000,10000,10000)
popt_interp1d = popt(x_wave)

dewar = interp1d(hsc_cam.dewar[:, 0], hsc_cam.dewar[:, 1], kind=3)
x_wave = np.linspace(4000,10000,10000)
dewar_interp1d = dewar(x_wave)

primary_reflect = interp1d(hsc_cam.primary_reflect[:, 0][0:311]*10, hsc_cam.primary_reflect[:, 1][0:311], kind=3)
x_wave = np.linspace(4000,10000,10000)
primary_reflect_interp1d = primary_reflect(x_wave)

combine_all = popt_interp1d * CCD_qe_interp1d * primary_reflect_interp1d * dewar_interp1d * 0.78

In [6]:
def compute_sn(redshift, wave_cen, filter_width, exptime, M_star, gal_r50, SED_model):

    # Initialize (pseudo-)random number generator
    random_seed = 1234567
    rng = galsim.BaseDeviate(random_seed)
    
    #distance luminosity in pc
    dl = cosmo_for_sn.Dl(0.0,redshift)*10**6 
    # value to go from L_sun/AA to erg/s/cm^2/AA at luminosity distance in pc
    to_cgs_at_dl = lsun / (4.0 * np.pi * (pc*dl)**2)
    
    # Generate the artificial narrowband filter (nb) with sigmoid damping
    def _sigmoid(x):
        return 1. / (1. + np.exp(-x))
    peak_response = combine_all[int(wave_cen)]
    wave_margin = 10
    half_width = int(filter_width / 2) + wave_margin
    wavelength = np.arange(half_width * 2) + wave_cen - half_width
    half_window = np.arange(half_width) - wave_margin
    response_curve = np.concatenate([_sigmoid(half_window), _sigmoid(half_window)[::-1]]) * peak_response
    filter_func = interp1d(wavelength, response_curve, kind=3)
    filter_nb = galsim.Bandpass(filter_func, wave_type='Ang',
                                blue_limit=min(wavelength), red_limit=max(wavelength)).withZeropoint(zeropoint='AB')
    
    
    # Read in galaxy SEDs from fsps (with emission lines)
    dwarf_seds = np.load('dwarf_sample_gaussian_test.npy')
    dwarf_wave = dwarf_seds['wave'][SED_model]
    dwarf_em = dwarf_seds['spec_em'][SED_model] #with emission lines
    #convert spec_em (in Lsun/Ang) into erg/s/cm^2/Ang at dl
    dwarf_em = dwarf_em*to_cgs_at_dl
    #spec_em is the spectra of a galaxy with stellar mass equal to dwarf_seds['mstar']
    #here convert it into M_star
    dwarf_em = dwarf_em/dwarf_seds['mstar'][SED_model]*10**M_star
    sed_func_em = interp1d(dwarf_wave, dwarf_em, kind=3)
    SED_dwarf_em = galsim.SED(sed_func_em, wave_type=units.AA, 
                           flux_type=units.erg/(units.s * units.cm**2)/units.AA)
    
    # Read in galaxy SEDs from fsps (continumm only)
    dwarf_ne = dwarf_seds['spec_ne'][SED_model] #continumm only
    #convert spec_ne (in Lsun/Ang) into erg/s/cm^2/Ang at dl
    dwarf_ne = dwarf_ne*to_cgs_at_dl
    #spec_ne is the spectra of a galaxy with stellar mass equal to dwarf_seds['mstar']
    #here convert it into M_star
    dwarf_ne = dwarf_ne/dwarf_seds['mstar'][SED_model]*10**M_star
    sed_func_ne = interp1d(dwarf_wave, dwarf_ne, kind=3)
    SED_dwarf_ne = galsim.SED(sed_func_ne, wave_type=units.AA, 
                           flux_type=units.erg/(units.s * units.cm**2)/units.AA)

    # Define the galaxy profile (here exponential)
    mono_gal = galsim.Exponential(half_light_radius=gal_r50)
    gal_SED_em = SED_dwarf_em.atRedshift(redshift)
    gal_em = mono_gal * gal_SED_em
    gal_SED_ne = SED_dwarf_ne.atRedshift(redshift)
    gal_ne = mono_gal * gal_SED_ne

    # Read in the sky spectra
    SED_sky = galsim.SED(spec_sky_func, wave_type=units.AA, 
                           flux_type=units.astrophys.photon/(units.s * units.m**2)/units.um)
    
    # Convolve the PSF
    PSF = galsim.Moffat(fwhm=0.6, beta=2.5)
    galfinal_em = galsim.Convolve([gal_em, PSF])
    galfinal_ne = galsim.Convolve([gal_ne, PSF])

    
    # Generate the image of the galaxy with emission lines
    # Draw profile through the filters
    img = galsim.ImageF(length, length, scale=pixel_scale)   
    if filter_number == 1:
        galfinal_em.drawImage(filter_r, image=img, area=area, exptime=exptime)
    if filter_number == 0:
        galfinal_em.drawImage(filter_nb, image=img, area=area, exptime=exptime)
    #add sky noise
    #sky level increases with exposure time
    if filter_number == 1:
        sky_level = SED_sky.calculateFlux(filter_r) * exptime #counts / arcsec^2
    if filter_number == 0:
        sky_level = SED_sky.calculateFlux(filter_nb) * exptime #counts / arcsec^2
    sky_level_pixel = sky_level * pixel_scale**2
    gain = 3. # HSC e- / ADU
    read_noise = 0.
    noise = galsim.CCDNoise(rng, gain=gain, read_noise=read_noise, sky_level=sky_level_pixel)
    img.addNoise(noise)
    # Save the image
    #Imageflux is photons/pixels
    out_filename = os.path.join('/Users/yifei/work/dwarf/computesn', 'computesn_em_1.fits')
    galsim.fits.write(img, out_filename)
    
    # Generate the image of the galaxy with continumm only
    # Draw profile through the filters
    img = galsim.ImageF(length, length, scale=pixel_scale)   
    if filter_number == 1:
        galfinal_ne.drawImage(filter_r, image=img, area=area, exptime=exptime)
    if filter_number == 0:
        galfinal_ne.drawImage(filter_nb, image=img, area=area, exptime=exptime)
    #add sky noise
    #sky level increases with exposure time
    if filter_number == 1:
        sky_level = SED_sky.calculateFlux(filter_r) * exptime #counts / arcsec^2
    if filter_number == 0:
        sky_level = SED_sky.calculateFlux(filter_nb) * exptime #counts / arcsec^2
    sky_level_pixel = sky_level * pixel_scale**2
    gain = 3. # HSC e- / ADU
    read_noise = 0.
    noise = galsim.CCDNoise(rng, gain=gain, read_noise=read_noise, sky_level=sky_level_pixel)
    img.addNoise(noise)
    # Save the image
    #Imageflux is photons/pixels
    out_filename = os.path.join('/Users/yifei/work/dwarf/computesn', 'computesn_ne_1.fits')
    galsim.fits.write(img, out_filename)
    
    return sky_level_pixel

In [7]:
def compute_completeness(points_number,redshift_cen,filter_width,exptime):
    detected=0
    for i in range(points_number):
        #filter center
        wave_cen = 6564*(1+redshift_cen) #AA
        #redshift range according to the filter width in AA
        reshift_range = filter_width/2/6564
        #redshift (central wavelength is computed from z)
        if redshift_cen - reshift_range >= 0.005 :
            redshift = random.uniform(redshift_cen-reshift_range,redshift_cen+reshift_range)
        if redshift_cen - reshift_range < 0.005:
            redshift = random.uniform(0.005,redshift_cen+reshift_range)            
        #log stellar mass
        M_star = random.uniform(8.0,9.0)
        #size : galaxy size
        gal_r50_Mpc = random.uniform(0.2,3.0)/1000 #0.2-3.0 kpc
        gal_r50 = gal_r50_Mpc/cosmo.angular_diameter_distance(redshift).value*(60*60*180/np.pi) #arcsec
        #SED model number (0-3999)
        SED_model = random.randint(0,3999)
    
        sky_level_pixel = compute_sn(redshift, wave_cen, filter_width, exptime, M_star, gal_r50, SED_model)
            
        data_em = fitsio.read("computesn_em_1.fits")
        data_ne = fitsio.read("computesn_ne_1.fits")
    
        # Do the photometry
        skynoise = np.sqrt(sky_level_pixel)
        flux_em, fluxerr_em, flag_em = sep.sum_circle(data_em, length/2, length/2, gal_r50/pixel_scale*5, err=skynoise, gain=1.0)
        #SNR_em = flux_em/fluxerr_em

        skynoise = np.sqrt(sky_level_pixel)
        flux_ne, fluxerr_ne, flag_ne = sep.sum_circle(data_ne, length/2, length/2, gal_r50/pixel_scale*5, err=skynoise, gain=1.0)

        SN_detection = (flux_em - flux_ne)/fluxerr_em

        if SN_detection>=4.0:
            detected=detected+1
                
        #print(i+1,detected,redshift,M_star,SN_detection)
        
    completeness = detected/(i+1)
    
    return completeness,reshift_range

In [8]:
def compute_SN_lensing(redshift_cen,reshift_range,completeness,survey_area):
    
    if redshift_cen - reshift_range >= 0.005 :
        z_lens_min = redshift_cen - reshift_range
    if redshift_cen - reshift_range < 0.005 :
        z_lens_min = 0.005
        
    z_lens_max = redshift_cen + reshift_range
    z_lens_mean = redshift_cen
    z_lens=np.atleast_1d([z_lens_mean])

    # Shot noise
    n_g = phiintegral * completeness
    g_shot_noise=1./n_g

    pk,kh=PS.class_pk(z_lens,cosmo_params=cosmo_params,pk_params=pk_params,return_s8=False)
    h=cosmo.h
    k=kh*h
    pk=pk/h**3
    rho=PS.Rho_crit(cosmo=cosmo)*cosmo.Om0

    x=HT_inv.k[2]>DS_prof['rp'].max()
    DS2=DS_prof['DS'][-1]*DS_prof['rp'][-1]/HT_inv.k[2][x]
    DS2=np.append(DS_prof['DS'],DS2)
    rp2=np.append(DS_prof['rp'],HT_inv.k[2][x]) #this doesnot help

    k2,p_gk2=HT_inv.projected_correlation(k_pk=rp2,pk=DS2,j_nu=2)
    p_gk2*=(2*np.pi)**2 #factors due to fourier convention

    b_g=1
    p_g=b_g**2*pk[0]
    p_gk=b_g*pk[0]*rho

    r_th,DS_th2=HT.projected_correlation(k_pk=k2,pk=p_gk2,j_nu=2)
    r_th,DS_th=HT.projected_correlation(k_pk=k,pk=p_gk,j_nu=2)

    rp,DS_th_b=HT.bin_mat(r=r_th,r_bins=rp_bins,mat=DS_th) #bin the theory predictions
    rp,DS_th_b2=HT.bin_mat(r=r_th,r_bins=rp_bins,mat=DS_th2) #bin the theory predictions

    L_W=cosmo.angular_diameter_distance(z_lens_max)-cosmo.angular_diameter_distance(z_lens_min)
    L_W=L_W.value

    area_comoving=survey_area*(np.pi/180)**2*cosmo.angular_diameter_distance(z_lens_mean)**2

    sigma_crit=PS.sigma_crit(zl=z_lens,zs=z_source,cosmo=cosmo)
    sigma_crit=sigma_crit[0,0].value

    d2r=np.pi/180.
    n_s=n_s_arcmin*3600/d2r**2
    shape_noise=sigma_gamma**2/n_s

    l,cl_kappa_kappa=PS.kappa_cl(zs1=z_source,p_zs1=[1],zs2=z_source,p_zs2=[1],zl_max=z_source,n_zl=100,
                             l=np.arange(5.e5),cosmo=cosmo,cl_z_func=PS.pk_l_z)
    chi=cosmo.angular_diameter_distance(z_lens)
    k_l=(l+0.5)/chi
    cl_intp=interp1d(k_l,cl_kappa_kappa,bounds_error=False,fill_value=0)

    l,cl_kappa_kappa=PS.kappa_cl(zs1=z_source,p_zs1=[1],zs2=z_source,p_zs2=[1],zl_max=z_source,n_zl=100,
                             l=np.arange(5.e5),cosmo=cosmo,cl_z_func=PS.pk_l_z)
    chi=cosmo.angular_diameter_distance(z_lens)
    k_l=(l+0.5)/chi
    cl_intp=interp1d(k_l,cl_kappa_kappa,bounds_error=False,fill_value=0)

    if shapenoiseonly == 1:
        p_kappa_kappa=sigma_crit**2*(cl_intp(k)*0+shape_noise)*chi**2 #shape noise only    
    else:
        p_kappa_kappa=sigma_crit**2*(cl_intp(k)*u.Mpc**2+shape_noise*chi**2)  #all noise .. for updated version (chi**2 is now inside brackets)
    
    taper_kw=dict({'large_k_lower':500,'large_k_upper':PS.pk_params['kmax'],'low_k_lower':PS.pk_params['kmin'],
                   'low_k_upper':PS.pk_params['kmin']*1.2})

    if shapenoiseonly == 1:
        r,cov_ggkk=HT.projected_covariance(k_pk=k,pk1=p_g*0+g_shot_noise,pk2=p_kappa_kappa,j_nu=2,taper=True,**taper_kw) #shape noise only
    else:
        r,cov_ggkk=HT.projected_covariance(k_pk=k,pk1=p_g+g_shot_noise,pk2=p_kappa_kappa,j_nu=2,taper=True,**taper_kw) #all noise


    r_re,cov_ggkk_re=HT.bin_cov(r=r,cov=cov_ggkk,r_bins=rp_bins)
    corr=HT.corr_matrix(cov=cov_ggkk_re)

    r,cov_gkgk=HT.projected_covariance(k_pk=k,pk1=p_gk,pk2=p_gk,j_nu=2,taper=True,**taper_kw)#return_Jrr=True,Jrr=Jrr
    r_re,cov_gkgk_re=HT.bin_cov(r=r,cov=cov_gkgk,r_bins=rp_bins)
    corr=HT.corr_matrix(cov=cov_gkgk_re)

    cov_ggkk_re/=(area_comoving.value*L_W)
    cov_gkgk_re/=area_comoving.value

    if shapenoiseonly == 1:
        cov_final=cov_ggkk_re #shape noise only    
    else:
        cov_final=(cov_ggkk_re+cov_gkgk_re)#/area_comoving.value #all noise

    corr=HT.corr_matrix(cov=cov_final)
    errors=HT.diagonal_err(cov=cov_final)
    errors_ggkk=HT.diagonal_err(cov=cov_ggkk_re)
    errors_gkgk=HT.diagonal_err(cov=cov_gkgk_re)

    noise_func = interp1d(r_re[2:41], errors_ggkk[2:41], kind=1)
    x_noise = np.linspace(r_re[7],r_re[40],100000)
    noise_interp1d = noise_func(x_noise)

    DS_error = noise_func(DS_prof['rp'][0:41])
    
    SN_lensing = DS_prof['DS']/DS_error #this should be modified to one number later!
    
    return SN_lensing

# Run Monte Carlo simulation

In [9]:
SN_lensing_array = []
redshift_cen_array = []
filter_width_array = []
exptime_array = []
completeness_array = []
survey_area_array = []

points_number = 200
redshift_cen = 0.03
completeness_limit = 0.8

redshift_cen_number = 0
while redshift_cen_number < 5:
    
    filter_width_number = 0
    filter_width = 100
    exptime = 300
    
    while filter_width_number < 20:
        print('redshift_cen =',redshift_cen)
        print('filter_width =',filter_width)
        print('exptime =', exptime)
        completeness,reshift_range = compute_completeness(points_number,redshift_cen,filter_width,exptime)
        print('completeness =',completeness)
        
        if completeness < completeness_limit:
            exptime = exptime + 300
            
        if exptime > 28800:
            filter_width_number = filter_width_number + 1
            
            SN_lensing_array.append(-999)
            redshift_cen_array.append(redshift_cen)
            filter_width_array.append(filter_width)
            exptime_array.append(exptime)
            completeness_array.append(completeness)

        if completeness > completeness_limit:
            survey_area = Nnights * imager_size * hours_night / (exptime/60/60)
            if survey_area >= 1000:
                survey_area = 1000
            print('survey_area =',survey_area)
            SN_lensing = compute_SN_lensing(redshift_cen,reshift_range,completeness,survey_area)
            print('SN_lensing =',SN_lensing[0])
            
            SN_lensing_array.append(SN_lensing)
            redshift_cen_array.append(redshift_cen)
            filter_width_array.append(filter_width)
            exptime_array.append(exptime)
            completeness_array.append(completeness)
            survey_area_array.append(survey_area)

            filter_width = filter_width + 40
            filter_width_number = filter_width_number + 1
            
    redshift_cen = redshift_cen + 0.01
    redshift_cen_number = redshift_cen_number + 1
    
np.savetxt('optimize_filter_SN_lensing_array_1.txt',np.array(SN_lensing_array).T,fmt='%.7f',delimiter=' ',newline='\n',footer='')
np.savetxt('optimize_filter_redshift_cen_array_1.txt',np.array(redshift_cen_array).T,fmt='%.7f',delimiter=' ',newline='\n',footer='')
np.savetxt('optimize_filter_filter_width_array_1.txt',np.array(filter_width_array).T,fmt='%.7f',delimiter=' ',newline='\n',footer='')
np.savetxt('optimize_filter_exptime_array_1.txt',np.array(exptime_array).T,fmt='%.7f',delimiter=' ',newline='\n',footer='')
np.savetxt('optimize_filter_completeness_array_1.txt',np.array(completeness_array).T,fmt='%.7f',delimiter=' ',newline='\n',footer='')
np.savetxt('optimize_filter_survey_area_array_1.txt',np.array(survey_area_array).T,fmt='%.7f',delimiter=' ',newline='\n',footer='')


redshift_cen = 0.03
filter_width = 100
exptime = 300
completeness = 1.0
survey_area = 1000


/Users/yifei/anaconda/lib/python3.6/site-packages/astropy/units/quantity.py:639: RuntimeWarning: divide by zero encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/yifei/work/dwarf/computesn/power_spectra.py:279: RuntimeWarning: divide by zero encountered in true_divide
  f=(l+0.5)**2/(l*(l+1.)) #correction from Kilbinger+ 2017
/Users/yifei/work/dwarf/computesn/hankel_transform.py:138: RuntimeWarning: invalid value encountered in double_scalars
  corr[i][j]=cov[i][j]/np.sqrt(cov[i][i]*cov[j][j])


SN_lensing = 2.5736281106048886
redshift_cen = 0.03
filter_width = 140
exptime = 300
completeness = 1.0
survey_area = 1000
SN_lensing = 3.0452545373267417
redshift_cen = 0.03
filter_width = 180
exptime = 300
completeness = 1.0
survey_area = 1000
SN_lensing = 3.453140272753258
redshift_cen = 0.03
filter_width = 220
exptime = 300
completeness = 0.995
survey_area = 1000
SN_lensing = 3.8082379901869996
redshift_cen = 0.03
filter_width = 260
exptime = 300
completeness = 1.0
survey_area = 1000
SN_lensing = 4.150641368470385
redshift_cen = 0.03
filter_width = 300
exptime = 300
completeness = 1.0
survey_area = 1000
SN_lensing = 4.458834439011537
redshift_cen = 0.03
filter_width = 340
exptime = 300
completeness = 0.995
survey_area = 1000
SN_lensing = 4.691418601769431
redshift_cen = 0.03
filter_width = 380
exptime = 300
completeness = 0.995
survey_area = 1000
SN_lensing = 4.821069790633315
redshift_cen = 0.03
filter_width = 420
exptime = 300
completeness = 0.99
survey_area = 1000
SN_lensing = 4

completeness = 0.99
survey_area = 1000
SN_lensing = 8.373273996237629
redshift_cen = 0.060000000000000005
filter_width = 380
exptime = 300
completeness = 0.955
survey_area = 1000
SN_lensing = 8.695073477882174
redshift_cen = 0.060000000000000005
filter_width = 420
exptime = 300
completeness = 0.95
survey_area = 1000
SN_lensing = 9.118236070985212
redshift_cen = 0.060000000000000005
filter_width = 460
exptime = 300
completeness = 0.98
survey_area = 1000
SN_lensing = 9.693124954697412
redshift_cen = 0.060000000000000005
filter_width = 500
exptime = 300
completeness = 0.955
survey_area = 1000
SN_lensing = 9.977298154547793
redshift_cen = 0.060000000000000005
filter_width = 540
exptime = 300
completeness = 0.935
survey_area = 1000
SN_lensing = 10.260950060779603
redshift_cen = 0.060000000000000005
filter_width = 580
exptime = 300
completeness = 0.865
survey_area = 1000
SN_lensing = 10.229920860673404
redshift_cen = 0.060000000000000005
filter_width = 620
exptime = 300
completeness = 0.935


ValueError: A value in x_new is above the interpolation range.

In [10]:
np.savetxt('optimize_filter_SN_lensing_array_1.txt',np.array(SN_lensing_array).T,fmt='%.7f',delimiter=' ',newline='\n',footer='')
np.savetxt('optimize_filter_redshift_cen_array_1.txt',np.array(redshift_cen_array).T,fmt='%.7f',delimiter=' ',newline='\n',footer='')
np.savetxt('optimize_filter_filter_width_array_1.txt',np.array(filter_width_array).T,fmt='%.7f',delimiter=' ',newline='\n',footer='')
np.savetxt('optimize_filter_exptime_array_1.txt',np.array(exptime_array).T,fmt='%.7f',delimiter=' ',newline='\n',footer='')
np.savetxt('optimize_filter_completeness_array_1.txt',np.array(completeness_array).T,fmt='%.7f',delimiter=' ',newline='\n',footer='')
np.savetxt('optimize_filter_survey_area_array_1.txt',np.array(survey_area_array).T,fmt='%.7f',delimiter=' ',newline='\n',footer='')


In [ ]:
points_number = 100
redshift_cen = 0.18
filter_width = 400
exptime = 300
completeness,redshift_range = compute_completeness(points_number,redshift_cen,filter_width,exptime)

In [ ]:
redshift_cen = 0.1
filter_width = 200
reshift_range = filter_width/2/6564
completeness = 0.8
survey_area = 500
SN_lensing = compute_SN_lensing(redshift_cen,reshift_range,completeness,survey_area)

In [ ]:
SN_lensing_array[2][0:4]

In [ ]:
r_sn_test = np.array([0.01,0.036,0.061,0.087])*1000
sn_test = np.array([17.9693947/0.3846234,6.8902154/0.1997475,3.3457646/0.1369632,2.6785669/0.1002576])
sn_test = SN_lensing_array[2][0:4]